# Set some environment variables

In [1]:
import json
import os
# Upload your spark_nlp_for_healthcare.json  to the default directory and then run this cell to authenticate 
with open('spark_nlp_for_healthcare.json', 'r') as f:
    for k, v in json.load(f).items():
        %set_env $k=$v

%set_env PYSPARK=3.4.2
%set_env SPARK_HOME=/home/ec2-user/SageMaker/spark-3.4.2-bin-hadoop3

env: SECRET=3.0.3-2dXXXXXXXXXXXXXXXXXXXXXXXXXX
env: SPARK_NLP_LICENSE=XXXXXXXXXXXXXXXX
env: AWS_ACCESS_KEY_ID=AKXXXXXXXXXXXXXXXX
env: AWS_SECRET_ACCESS_KEY=qDRXXXXXXXXXXXXXXXXXXXXXXX
env: JSL_VERSION=3.0.3
env: PUBLIC_VERSION=3.0.3
env: PYSPARK=3.0.2
env: SPARK_HOME=/home/ec2-user/SageMaker/spark-3.0.2-bin-hadoop2.7


# Download packages

In [2]:
# 1 line install for sagemaker
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_sagemaker_setup.sh
!bash jsl_sagemaker_setup.sh

--2021-07-01 18:06:47--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_sagemaker_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1476 (1.4K) [text/plain]
Saving to: ‘jsl_sagemaker_setup.sh’

jsl_sagemaker_setup 100%[===================>]   1.44K  --.-KB/s    in 0s      

2021-07-01 18:06:47 (22.5 MB/s) - ‘jsl_sagemaker_setup.sh’ saved [1476/1476]

setup SageMaker for PySpark 3.0.2 and Spark NLP 3.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker-pyspark 1.4.2 requires pyspark==2.4.0, but you have pyspark 3.0.2 which is incompatible.
You should consider upgrading via the '/home/ec2-u

In [3]:
# Imports
import sparknlp_jsl
from sparknlp_jsl.annotator import *
from sparknlp_jsl import start
from pyspark.ml import PipelineModel
from sparknlp_jsl.annotator import *
from sparknlp.base import *

# Start authenticated sparksession for NLP-Healthcare
spark = start(os.environ["SECRET"])

# Download a simple model to test

In [4]:
SentenceEntityResolverModel.pretrained("sbiobertresolve_rxcui" , "en", "clinical/models")

sbiobertresolve_rxcui download started this may take some time.
Approximate size to download 53.2 MB
[OK!]


ENTITY_57d31cc0027b

# Download something more interesting (this will require at least an ml.t3.xlarge instance)

In [5]:
# Sample Healthcare pipe  
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")
    
umls_resolver = SentenceEntityResolverModel.pretrained("sbiobertresolve_umls_major_concepts", "en", "clinical/models") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

umls_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        umls_resolver])


umls_lp = LightPipeline(umls_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_umls_major_concepts download started this may take some time.
Approximate size to download 825.5 MB
[OK!]


In [6]:
umls_lp
text = 'type two diabetes mellitus'
umls_lp.annotate(text)

{'ner_chunk': ['type two diabetes mellitus'],
 'sbert_embeddings': ['type two diabetes mellitus'],
 'umls_code': ['C4014362']}

In [7]:
!java -version

openjdk version "1.8.0_282"
OpenJDK Runtime Environment (build 1.8.0_282-b08)
OpenJDK 64-Bit Server VM (build 25.282-b08, mixed mode)
